In [54]:
import nltk
import random
import numpy as np
import string

## Corpus definition

In [55]:
file = open('corpusFiles/Corpus_crucero.txt')
raw = file.read()
raw

'Para saber cómo vestir, la estación del año y la temperatura media de la zona marcan siempre la vestimenta más adecuada para los cruceros, aunque los cruceros tienden a ser cada vez más informales sobre todo durante el día.\n\nEl programa del día le indicará siempre la ropa más adecuada.\n\nRespecto al calzado, en tierra necesitarás zapatos cómodos para caminar, así como sombrero y protección solar (en verano).\n\nLas propinas son una norma generalizada, obligatorias, en todos los barcos de crucero, de hecho, forman parte del sueldo del personal y forman parte del llamado servicio.\n\nDependiendo de la naviera, para embarcar es conveniente llegar al embarque 3 o 4 horas antes de la hora de zarpar, normalmente los embarques suelen ser al mediodía y casi todas las compañías de cruceros ofrecen un buffet para comer.\n\nLa comida es un punto fuerte, los cruceros son una oportunidad única para poder degustar una amplia gama de alta calidad, con una diversidad de platos típicos de todas las

### Text Preprocessing

In [56]:
raw = raw.lower()

sent_tokens = nltk.sent_tokenize(raw)  # we separate text in sentences/phrases
word_tokens = nltk.word_tokenize(raw)  # separate text in single tokens
lemmer = nltk.stem.WordNetLemmatizer()


def lem_tokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]


remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def lem_normalize(text):
    return lem_tokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

## now we are going to evaluate the similarity of the user message

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords


def response(user_response: str):
    sent_tokens.append(user_response)  ## we add to corpus the final user response

    t_fiz_vect = TfidfVectorizer(tokenizer=lem_normalize, stop_words=stopwords.words('spanish'))
    t_fiz_vect = t_fiz_vect.fit_transform(sent_tokens)

    #now we evaluate the similarity between user message (t_fiz_vect[:-1]) and the corpus(t_fiz_vec)
    vals = cosine_similarity(t_fiz_vect[-1], t_fiz_vect)
    idx = vals.argsort()[0][-2]

    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]

    if req_tfidf == 0:
        return 'I`m sorry, I dont understand you. If I can`t answer your question contact with the page team'

    return sent_tokens[idx]

## manual coincidences definition

In [58]:
greetings_inputs = ('hola', 'buenas', 'saludos', 'qué tal', 'hey', 'buenos dias')
greetings_responses = ['hola', 'buenas', 'saludos', 'qué tal', 'hey', 'buenos dias', 'Cómo puedo ayudarte?',
                       'hola encantado de atenderte']


def greetings(sentence):
    for word in sentence.split():
        if word.lower() in greetings_inputs:
            return random.choice(greetings_responses)

### Manual response generation

In [61]:
print('Hi, im robot, in what i can help you?')

while True:
    user_response = input().lower()
    print(user_response)

    if user_response == 'salir':
        break

    greeting = greetings(user_response)

    if greeting is not None:
        print(greeting)
        continue

    print('Robot:', end='')
    print(response(user_response))
    sent_tokens.remove(user_response)

Hi, im robot, in what i can help you?
hola
qué tal
que tal
Robot:pipas, cigarrillos y puros están permitidos sólo en algunos lugares dedicados a tal fin.
1 32
tengo un problema gordo, he roto el barco
Robot:la reserva incluye el alojamiento en el barco en la categoría elegida, el transporte marítimo, todas las comidas, algunas bebidas sin alcohol (dependiendo del barco) y la mayoría del entretenimiento.
1 32
la he liado muchisimo
Robot:I`m sorry, I dont understand you. If I can`t answer your question contact with the page team
1 32
salir
